In [1]:
# Imports 
import os
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

os.environ["pyna_language"] = 'python'
from pyNA.pyna import pyna

In [2]:
# Inputs 
z_cb_lst = np.hstack((np.linspace(25, 250, 10), np.linspace(300, 750, 10)))

TS_cb_lst = dict()
TS_cb_lst['stca']= dict()
TS_cb_lst['stca']['V2'] = np.linspace(0.75, 1.0, 11)
TS_cb_lst['stca']['Vmax'] = np.linspace(0.4, 1.0, 25)
TS_cb_lst['a10'] = dict()
TS_cb_lst['a10']['V2'] = np.linspace(0.775, 1.0, 10)
TS_cb_lst['a10']['Vmax'] = np.linspace(0.675, 1.0, 14)

In [4]:
epnl = dict()

for ac_name in ['stca', 'a10']:
    
    epnl[ac_name] = dict()

    for case in ['V2', 'Vmax']:
    
        epnl[ac_name][case] = dict()
    
        # Start of sideline noise 
        if ac_name == 'stca' and case == 'V2':
            start = 28
        elif ac_name == 'stca' and case == 'Vmax':
            start = 46
        elif ac_name == 'a10' and case == 'V2':
            start = 64
        elif ac_name == 'a10' and case == 'Vmax':
            start = 76
            
        epnl[ac_name][case]['flyover'] = np.load('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/epnl_flyover.npy') 
        epnl[ac_name][case]['sideline'] = np.load('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/epnl_sideline.npy')
        epnl[ac_name][case]['lateral'] = np.max(epnl[ac_name][case]['sideline'][:,:,start:], axis=2)
        

In [5]:
r_min_f = dict()
r_min_l = dict()
Vj_100 = dict()
Fsp_100 = dict()
Fsp_cb = dict()

Lsrc_100 = dict()
Lsrc_cb = dict()
Lprop_l = dict()
Lprop_f = dict()
Ltot = dict()

for ac_name in ['stca', 'a10']:
    
    r_min_f[ac_name] = dict()
    r_min_l[ac_name] = dict()
    Vj_100[ac_name] = dict()
    Fsp_100[ac_name] = dict()
    Fsp_cb[ac_name] = dict()
    
    Lsrc_100[ac_name] = dict()
    Lsrc_cb[ac_name] = dict()
    Lprop_l[ac_name] = dict()
    Lprop_f[ac_name] = dict()
    Ltot[ac_name] = dict()
    
    for case in ['V2', 'Vmax']:
                
        TS = 1.0
        z_cb = 700.0
                
        traj = pd.read_csv('../cases/'+ac_name+'/trajectory/STCB/'+case+'/trajectory_' + ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv')
        eng  = pd.read_csv('../cases/'+ac_name+'/engine/STCB/'+case+'/engine_' + ac_name + '_' + str(np.round(z_cb,3)) + '_' + str(np.round(TS,3)) + '.csv')
        
        r_f = np.sqrt((traj['X [m]']-6500)**2+(traj['Z [m]'])**2)
        r_min_f[ac_name][case] = np.min(r_f)
        
        i_max_lst = np.argmax(epnl[ac_name][case]['sideline'][-1,-1, :])
        
        r_l = np.sqrt((traj['X [m]']-np.linspace(0, 6500, 131)[i_max_lst])**2+450**2+(traj['Z [m]'])**2)
        r_min_l[ac_name][case] = np.min(r_l)
        
        i_M025 = np.where(traj['M_0 [-]']>0.25)[0][0]
        
        Vj_100[ac_name][case] = eng['Jet V [m/s]'][i_M025] 
        Fsp_100[ac_name][case] = (Vj_100[ac_name][case] - 0.25*340.294)/340.294
        
        Fsp_cb[ac_name][case] = Fsp_100[ac_name][case] - 0.7*(1-TS_cb_lst[ac_name][case][0])
        
        Lsrc_100[ac_name][case] = 10*np.log10((Fsp_100[ac_name][case]*340.294)**8)
        Lsrc_cb[ac_name][case] = 10*np.log10((Fsp_cb[ac_name][case]*340.294)**8)
        Lprop_l[ac_name][case] = 10*np.log10(1/r_min_l[ac_name][case]**2)
        Lprop_f[ac_name][case] = 10*np.log10(1/r_min_f[ac_name][case]**2)

In [9]:
def error(x, Lsrc_100, Lprop_l, epnl):
    
    e = 0

    for ac_name in ['stca', 'a10']:    
        for case in ['V2', 'Vmax']:
    
            e += (x[0]*Lsrc_100[ac_name][case] + x[1]*Lprop_l[ac_name][case]) - epnl[ac_name][case]['lateral'][-1,-1]

            print((x[0]*Lsrc_100[ac_name][case] + x[1]*Lprop_l[ac_name][case]))
            print(epnl[ac_name][case]['lateral'][-1,-1])
            print('---')
    
    return e

In [10]:
import scipy
scipy.optimize.minimize(error, x0=[1,1], args=(Lsrc_100, Lprop_l, epnl), method='SLSQP')

148.80540346471727
98.73524139671633
---
148.8343682983935
97.54937707224582
---
147.05558673758102
99.56925349941731
---
147.04672996705474
99.26282061228927
---
148.80540648127328
98.73524139671633
---
148.83437131500685
97.54937707224582
---
147.05558973085445
99.56925349941731
---
147.04673296028412
99.26282061228927
---
148.80540266553456
98.73524139671633
---
148.83436749958508
97.54937707224582
---
147.05558593560662
99.56925349941731
---
147.0467291649924
99.26282061228927
---
-174667.51045160863
98.73524139671633
---
-174665.18668257544
97.54937707224582
---
-173449.18931200315
99.56925349941731
---
-173448.0824901458
99.26282061228927
---
-174667.51044859207
98.73524139671633
---
-174665.18667955883
97.54937707224582
---
-173449.18930900987
99.56925349941731
---
-173448.08248715257
99.26282061228927
---
-174667.51045240782
98.73524139671633
---
-174665.18668337425
97.54937707224582
---
-173449.18931280513
99.56925349941731
---
-173448.08249094788
99.26282061228927
---
-104875

     fun: -1.1146444004218292e+21
     jac: array([ 806.62654835, -214.88445047])
 message: 'Rank-deficient equality constraint subproblem HFTI'
    nfev: 75
     nit: 25
    njev: 25
  status: 7
 success: False
       x: array([-1.29029300e+18,  3.43718757e+17])

In [133]:
w = [0.8, 1]

for ac_name in ['stca', 'a10']:    
    for case in ['V2', 'Vmax']:
        
        print(w[0]*Lsrc_100[ac_name][case] + w[1]*Lprop_l[ac_name][case])

108.31787391169055
108.34606943632468
106.88055100395695
106.87228529136792


In [8]:
for ac_name in ['stca', 'a10']:    
    for case in ['V2', 'Vmax']:
        print(epnl[ac_name][case]['lateral'][-1,-1])

98.73524139671633
97.54937707224582
99.56925349941731
99.26282061228927
